In [2]:
import re  # For preprocessing
import pandas as pd  # For data handling
from time import time  # To time our operations
from collections import defaultdict  # For word frequency
import pandas as pd
import spacy  # For preprocessing

import logging  # Setting up the loggings to monitor gensim
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)

In [3]:
stop_words=[
    'من','إلى','عن','على','في',
    'بن','التي','الذي','الذين','له','لها','هذه','هذا'
    ,'به','بها','عبر','وإن','ليست','حول'
    ,'أين','بل','بين','ما','أو','لا','لو'
    ,'إذ','أن','لو','كمن','ليس','وليس','وليسوا','ليسو','ليسوا','وليسو'
    ,'أين','وأين','اين','إلا','وإلا','ولو','لنا','قد','وقد','ومن','فمن','منكم','منهم','منهن','منها','فكأن','كأنه','فكأنه','كم','ذاك','إذا','منه'
]

waw_words=['وزارة','وفاء','ولاء','ويل']
lam_words=['لين','ليت','لون','لنت']
baa_words=['برق','براك','براق','','']
faa_words=['فناء','فنى','فنائ','فضل' ,'فضلتهم','وفضلتهم' , 'وفضلت' , 'فضلت']
c_words=['مكة','الله']

In [5]:
data = pd.read_csv('./abbasi/abbasi_review.csv', encoding='utf-8')
sample_data='رحبت دولة الإمارات العربية المتحدة بدعوة خادم الحرمين الشريفين الملك سلمان بن عبدالعزيز آل سعود لأشقائه قادة دول مجلس التعاون الخليجي والدول العربية لعقد قمتين خليجية وعربية طارئتين بمكة المكرمة يوم الخميس 30 مايو الحالي للتشاور والتنسيق حول الظروف الدقيقة التي تمر بها المنطقة وتداعياتها على الأمن والاستقرار.وقالت وزارة الخارجية والتعاون الدولي في بيان لها إن هذه المبادرة ليست بالغريبة من القيادة السعودية التي طالما حرصت وعملت على ترسيخ الأمن والسلام في المنطقة.. وأشادت في هذا السياق بالدور القيادي لخادم الحرمين الشريفين الملك سلمان بن عبدالعزيز آل سعود في كل ما يجمع الكلمة ويوحد الصف وينسق المواقف.وأضافت وزارة الخارجية والتعاون الدولي في بيانها إن الظروف الدقيقة الحالية تتطلب موقفا خليجيا وعربيا موحدا في ظل التحديات والأخطار المحيطة وإن وحدة الصف ضرورية وإن الدعوة الكريمة التي بادر بها خادم الحرمين الشريفين تمثل فرصة مهمة لدول المنطقة لتحقيق ما تصبو إليه من تعزيز فرص الاستقرار والسلام والتصدي للتحديات والأخطار المحيطة عبر موقف خليجي وعربي جماعي يحرص على أمننا المشترك وسيادتنا وإنجازاتنا.'
data.head()


,0,1,name,poet,review
0,سل الملك الكريم إلام تبني,وأين؟ وقد تجاوزتَ السماءَ,الشريف المرتضى,\r\nسل الملك الكريم إلام تبني,0
1,أجِدَّك لا براك اللَّه إلا,علاءً أو عطاءً أو وفاء,الشريف المرتضى,\r\nسل الملك الكريم إلام تبني,0
2,ولو ذوبتني ما كنت إلا,ولاءً أو دعاءً أو ثناء,الشريف المرتضى,\r\nسل الملك الكريم إلام تبني,0
3,منحتك من سواء الصدروداً,يكاد لفرطه يروي الظماءَ,الشريف المرتضى,\r\nسل الملك الكريم إلام تبني,0
4,أيعجزني إذا احتكُّوا هِناءٌ,وللكلبى إذا مرضوا شفاءَ,الشريف المرتضى,\r\nسل الملك الكريم إلام تبني,0


In [4]:
def clean(jomla):
    jomla = re.sub('[^ء-ي ]+', '', jomla)
    jomla = re.sub(' +', ' ', jomla)
    return jomla

print(clean(data['0'][0]))


سل الملك الكريم إلام تبني


In [5]:
sentences=[]
for i in range(len(data['0'])):
    try:
        sentence1=data['0'][i]
        sentence2=data['1'][i]

        sentence1=clean(sentence1)
        sentence2=clean(sentence2)

        sentences.append(sentence1.split(' '))
        sentences.append(sentence2.split(' '))
    except:
        continue
    
sentences[:5]

[['سل', 'الملك', 'الكريم', 'إلام', 'تبني'],
 ['', 'وأين', 'وقد', 'تجاوزت', 'السماء'],
 ['أجدك', 'لا', 'براك', 'الله', 'إلا'],
 ['', 'علاء', 'أو', 'عطاء', 'أو', 'وفاء'],
 ['ولو', 'ذوبتني', 'ما', 'كنت', 'إلا']]

In [6]:
words_list=[]
for s in sentences:
    for w in s:
        words_list.append(w)
        
len(words_list)
    

2598687

In [7]:
def is_begin_with_al(word):
    if word[0] == 'ا' and word[1] == 'ل':
        return (True,2)
    if word[1] == 'ا' and word[2] == 'ل':
        return (True,3)
    else:
        return False

In [8]:
words_list_refined=[]
word_history={
    'al':0,
    'fl':0,
    'bl':0,
    'wl':0,
    'll':0,
    'lee':0,
    'waa':0,
    'baa':0,
    'faa':0,
    'at':0,
    'woon':0,
    'haa':0,
    'naa':0,
    'aa':0
            }
for word in words_list:
    try:
        if word in stop_words:
            word=''
        if word in c_words:
            words_list_refined.append(word)
            continue
            
        if word[-1] == 'ا' and len(word[:-1]) >= 3:
            word=word[:-1]
            word_history.update({'aa':1})
        
        
        if word[-2] == 'ن' and word[-1]=='ي' and len(word[:-2]) >= 3:
            if (is_begin_with_al(word) == (True,2) and len(word[2:-2]) >= 4 ) or (is_begin_with_al(word) == (True,3) and len(word[3:-2]) >= 4 ) :
                word=word[:-2]
            if is_begin_with_al(word) == False:
                word=word[:-2]
        if word[-1] == 'ي':
            word=word[:-1]
            
        if word[-1] == 'ه':
            word=word[:-1]
            word_history.update({'haa':1})
        
            
        if word[0] == 'و' and word not in waw_words:
            word=word[1:]
            word_history.update({'waa':1})
            
        if word[0] == 'ف' and word not in faa_words :
            word=word[1:]
            word_history.update({'faa':1})
            
        if word[0] == 'ب' and word not in baa_words and len(word) >=4:
            word=word[1:]
            word_history.update({'baa':1})
            
        if word[0] == 'ل' and word[1]=='ل' :
            word=word[2:]
            word_history.update({'ll':1})
        if word[0] == 'ل' and word not in lam_words and word not in stop_words:
            word=word[1:]
            word_history.update({'lee':1})
        if word[0] == 'ا' and word[1]=='ل' and word not in c_words :
            word=word[2:]
            word_history.update({'al':1})
        if word[0] == 'و' and word[1]=='ا' and word[2] =='ل':
            word=word[3:]
            word_history.update({'wl':1})
        #if word[-1] == 'ة':
        #    word=word[:-1]

        if word[-2] == 'ه' and word[-1]=='ا' :
            word=word[:-2]
            word_history.update({'haa':1})
        if word[-2] == 'ي' and word[-1]=='ن' and word not in stop_words and len(word) >=4 :
            if word[-3]=='ت':
                word=word[:-3]+'ة'
            else:
                word=word[:-2]
        if word[-2] == 'ن' and word[-1]=='ا' :
            
            if len(word[:-2] <=3 ):
                word[-1]=='ى' 
            elif word[-3]=='ت':
                word=word[:-3]+'ة'
            else:
                word=word[:-2]
        if word[-2] == 'و' and word[-1]=='ن' and len(word) > 3:
            word=word[:-2]
        if word[-2] == 'ا' and word[-1]=='ت' :
            word=word[:-2]
            

        if word[-1] == 'ئ':
            word=word[:-1] +'ء'
    except:
        a=1
       # print ('error')
    words_list_refined.append(word)
len(words_list_refined)


2598687

In [9]:
word='أحمد'
len(word)
print (len(word) <3)

False


In [10]:
ba_word=[]
for i in range(len(words_list)):
    row={'before':words_list[i],'after':words_list_refined[i]}
    ba_word.append(row)

df=pd.DataFrame(ba_word)
df 

,after,before
0,سل,سل
1,ملك,الملك
2,كريم,الكريم
3,إلام,إلام
4,تبن,تبني
5,,
6,,وأين
7,,وقد
8,تجاوزت,تجاوزت
9,سماء,السماء


In [11]:
df[0:30]

,after,before
0,سل,سل
1,ملك,الملك
2,كريم,الكريم
3,إلام,إلام
4,تبن,تبني
5,,
6,,وأين
7,,وقد
8,تجاوزت,تجاوزت
9,سماء,السماء


In [12]:
df[30:60]

,after,before
30,,أو
31,ثناء,ثناء
32,منحتك,منحتك
33,,من
34,سواء,سواء
35,صدرود,الصدرودا
36,,
37,يكاد,يكاد
38,فرط,لفرطه
39,يرو,يروي


In [13]:
df[60:90]

,after,before
60,ندى,ندى
61,فضلت,وفضلت
62,مال,مالا
63,,
64,,ومن
65,طلب,طلب
66,ثناء,الثناء
67,رمى,رمى
68,ثراء,الثراء
69,أمن,أمن


In [14]:
df[90:120]

,after,before
90,,له
91,قدماك,قدماك
92,تسعى,تسعى
93,,
94,,إلى
95,أعواد,أعواده
96,,أو
97,قيل,قيل
98,جاء,جاء
99,برق,برق


In [15]:
df[120:150]

,after,before
120,,من
121,نور,نوره
122,,بل
123,ماء,مائه
124,رواء,وروائه
125,ماء,والماء
126,,بين
127,مصندل,مصندل
128,مكفر,ومكفر
129,,


In [16]:
df[150:180]

,after,before
150,يهد,يهدي
151,,لنا
152,نفح,نفحاته
153,,من
154,ماء,مائه
155,زمن,زمن
156,ربيع,الربيع
157,جلبت,جلبت
158,أزكى,أزكى
159,متجر,متجر


In [17]:
df[180:210]

,after,before
180,مجتو,والمجتوي
181,هو,هو
182,هارب,هارب
183,ذماء,بذمائه
184,,ما
185,بحر,البحر
186,,في
187,تزخار,تزخاره
188,غيث,والغيث
189,,في


In [18]:
df[210:240]

,after,before
210,عصر,عصره
211,,
212,ممتدح,ممتدحين
213,مدح,بمدحه
214,ثناء,وثنائه
215,يا,يا
216,,من
217,يل,يلي
218,أمر,أمر
219,قضاء,القضاء


In [19]:
df[240:270]

,after,before
240,,
241,ت,ت
242,غنا,غناه
243,,في
244,,ذاك
245,وعاء,الوعاء
246,رب,ولرب
247,ثكلى,ثكلى
248,,قد
249,تركـ,تركـ


In [20]:
df[270:300]

,after,before
270,,
271,ألبس,وألبس
272,بيد,البيد
273,ظلماء,والظلماء
274,يلب,واليلبا
275,أترك,وأترك
276,خود,الخود
277,معسول,معسولا
278,مقبل,مقبلها
279,,


In [21]:
df[300:330]

,after,before
300,,
301,,إذا
302,مشت,مشت
303,هلال,وهلال
304,شهر,الشهر
305,منتقب,منتقبا
306,تظل,تظل
307,تنثر,تنثر
308,,من
309,أجفان,أجفانها


In [22]:
df[1200:1230]

,after,before
1200,يهب,يهبه
1201,أراحن,أراحنا
1202,الله,الله
1203,منك,منك
1204,يا,يا
1205,زمن,زمنا
1206,,
1207,أرعن,أرعن
1208,يصطاد,يصطاد
1209,صقر,صقره
